In [1]:
import os
from dotenv import load_dotenv
import python_bithumb

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기
BITHUMB_API_KEY = os.getenv("BITHUMB_API_KEY")
BITHUMB_SECRET_KEY = os.getenv("BITHUMB_SECRET_KEY")

In [ ]:
import os
import pandas as pd
import python_bithumb
import matplotlib.pyplot as plt
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

# 환경 변수 가져오기 (실제 사용 시 .env 또는 OS 환경변수 설정)
BITHUMB_API_KEY = os.getenv("BITHUMB_API_KEY")
BITHUMB_SECRET_KEY = os.getenv("BITHUMB_SECRET_KEY")
SLACK_BOT_TOKEN = os.getenv("SLACK_BOT_TOKEN")
CHANNEL_NAME = os.getenv("CHANNEL_NAME")

client = WebClient(token=SLACK_BOT_TOKEN)  # 직접 토큰 입력 (환경 변수 대신)

access_key = BITHUMB_API_KEY 
secret_key = BITHUMB_SECRET_KEY  

bithumb = python_bithumb.Bithumb(access_key, secret_key)

# 예시: 원화 잔고 조회
krw_balance = bithumb.get_balance("KRW")

def send_message(message: str, channel: str = CHANNEL_NAME, image_path: str = None):
    try:
        client.chat_postMessage(channel=channel, text=message)
        print("메시지 전송 성공")
    except SlackApiError as e:
        print(f"Slack API 오류: {e.response['error']}")
    except Exception as e:
        print(f"오류 발생: {e}")



# (B) 짧은 봉(스캘핑용) CSV 저장
def save_scalping_ohlcv_to_csv(
    ticker: str = "KRW-BTC",
    interval: str = "minute1",
    count: int = 100,
    filename: str = "scalping_data.csv"
):
    df = python_bithumb.get_ohlcv(ticker, interval=interval, count=count)
    if df is None or df.empty:
        print(f"[WARNING] {ticker} {interval} 데이터가 비어 있습니다. 저장을 중단합니다.")
        return
    
    # 폴더 자동 생성
    dir_name = os.path.dirname(filename)
    if dir_name and not os.path.exists(dir_name):
        os.makedirs(dir_name, exist_ok=True)
    
    df.to_csv(filename, encoding='utf-8-sig', index=True)
    print(f"[INFO] '{ticker}' {interval} 데이터 {count}개를 '{filename}'에 저장했습니다.")

import matplotlib.dates as mdates

def plot_ohlcv_from_csv(csv_path: str, output_folder : str="img", out_image_path: str = "ohlcv_plot.png") -> str:
    # 결과 폴더 생성
    os.makedirs(output_folder, exist_ok=True)
    out_image_path = os.path.join(output_folder, out_image_path)

    if not os.path.exists(csv_path):
        print(f"[ERROR] CSV 파일 '{csv_path}'이(가) 존재하지 않습니다.")
        return ""
    
    # CSV 읽기
    df = pd.read_csv(csv_path, index_col=0, parse_dates=True)
    if df.empty:
        print(f"[WARNING] CSV 파일 '{csv_path}'이(가) 비어 있습니다.")
        return ""
    
    # Matplotlib으로 'Open' 값만 시계열 그래프로 그리기
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df['open'], label='Open Price', color='blue')
    
    # 그래프 꾸미기
    plt.title(f"Open Price")
    plt.xlabel("DateTime")
    plt.ylabel("Price (KRW)")
    plt.legend()

    # x축 눈금을 촘촘하게 설정
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=5))  # 5분 간격으로 주요 눈금
    ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=1))  # 1분 간격으로 보조 눈금
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))  # HH:MM 형식으로 표시
    plt.xticks(rotation=45)  # x축 눈금 기울이기
    
    # 촘촘한 Grid 설정
    plt.grid(which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.minorticks_on()  # 보조 눈금 활성화
    
    # 이미지 파일 저장
    plt.savefig(out_image_path, dpi=150, bbox_inches='tight')
    plt.close()  # 메모리 해제
    
    print(f"[INFO] 그래프를 '{out_image_path}'로 저장했습니다.")
    return out_image_path



filename5 = "5min.csv"
filename15 = "15min.csv"
# 1) OHLCV CSV 저장
save_scalping_ohlcv_to_csv(
    ticker="KRW-BTC",
    interval="minute5",
    count=30,
    filename=filename5
)

save_scalping_ohlcv_to_csv(
    ticker="KRW-BTC",
    interval="minute15",
    count=15,
    filename=filename15
)

# 2) 저장된 CSV 기반으로 그래프 PNG 생성
plot_ohlcv_from_csv(
    csv_path=r"D:\바탕화면\code\Stock\Code\5min.csv",
    out_image_path="5min.png"
)
plot_ohlcv_from_csv(
    csv_path=r"D:\바탕화면\code\Stock\Code\15min.csv",
    out_image_path="15min.png"
)

[INFO] 'KRW-BTC' minute5 데이터 30개를 '5min.csv'에 저장했습니다.
[INFO] 'KRW-BTC' minute15 데이터 15개를 '15min.csv'에 저장했습니다.
[INFO] 그래프를 'img\5min.png'로 저장했습니다.
[INFO] 그래프를 'img\15min.png'로 저장했습니다.


'img\\15min.png'

In [7]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates  # Matplotlib 날짜 포맷터 및 로케이터

def save_and_plot_ohlcv(
    ticker: str = "KRW-BTC",
    interval: str = "minute5",
    count: int = 100,
    csv_filename: str ="",
    img_filename: str = "img/scalping_plot.png"
):
    # OHLCV 데이터 가져오기
    df = python_bithumb.get_ohlcv(ticker, interval=interval, count=count)
    if df is None or df.empty:
        print(f"[WARNING] {ticker} {interval} 데이터가 비어 있습니다. 작업을 중단합니다.")
        return
    
    csv_filename = "csv/" + csv_filename + ".csv"

    # CSV 저장
    csv_dir = os.path.dirname(csv_filename)
    if csv_dir and not os.path.exists(csv_dir):
        os.makedirs(csv_dir, exist_ok=True)
    
    df.to_csv(csv_filename, encoding='utf-8-sig', index=True)
    print(f"[INFO] '{ticker}' {interval} 데이터 {count}개를 '{csv_filename}'에 저장했습니다.")

    img_filename = "img/" + img_filename + ".png"
    # 이미지 저장
    img_dir = os.path.dirname(img_filename)
    if img_dir and not os.path.exists(img_dir):
        os.makedirs(img_dir, exist_ok=True)

    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df['open'], label='Open Price', color='blue')

    # 그래프 꾸미기
    plt.title(f"{ticker} Open Price ({interval})")
    plt.xlabel("DateTime")
    plt.ylabel("Price (KRW)")
    plt.legend()

    # x축 눈금 설정
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=5))
    ax.xaxis.set_minor_locator(mdates.MinuteLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.xticks(rotation=45)

    # Grid 설정
    plt.grid(which='both', linestyle='--', linewidth=0.5, alpha=0.7)
    plt.minorticks_on()

    plt.savefig(img_filename, dpi=150, bbox_inches='tight')
    plt.close()
    print(f"[INFO] 그래프를 '{img_filename}'에 저장했습니다.")

save_and_plot_ohlcv(count = 30, csv_filename="5min", img_filename="5min")


[INFO] 'KRW-BTC' minute5 데이터 30개를 'csv/5min.csv'에 저장했습니다.
[INFO] 그래프를 'img/5min.png'에 저장했습니다.


In [1]:
from main import Coin_Bot  # 클래스 이름과 파일명이 일치하도록 수정
from dotenv import load_dotenv
import os

load_dotenv()  # 환경 변수 로드

# 환경 변수에서 Slack과 Google API 키 가져오기
SLACK_TOKEN = os.getenv("SLACK_BOT_TOKEN")
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")

# 필수 인자를 전달하여 CoinNewsBot 객체 생성
coin_bot = Coin_Bot(slack_token=SLACK_TOKEN, google_api_key=GOOGLE_API_KEY)

coin_bot.save_and_plot_ohlcv(interval="minute5", count=15, csv_filename="15min", img_filename="15min")


[INFO] 데이터 15개를 'csv/15min.csv'에 저장했습니다.
[INFO] 그래프를 'img/15min.png'에 저장했습니다.


In [ ]:
coin_bot.run_autoupdate()

[INFO] 자동 업데이트 시작 (5분 간격)

[INFO] 뉴스 확인 중... (2025-01-27 20:45:23)
[INFO] Slack 메시지가 전송되었습니다.
[INFO] 5분 뒤 다음 업데이트 진행...


In [ ]:
coin_bot.run_autoupdate()

[INFO] 자동 업데이트 시작 (5분 간격)

[INFO] 뉴스 확인 중... (2025-01-27 21:09:15)
[INFO] Slack 메시지가 전송되었습니다.
[INFO] 5분 뒤 다음 업데이트 진행...
